Check the GPU and Memory

In [8]:
!/opt/bin/nvidia-smi

Fri Jul 10 09:58:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   65C    P0    40W / 250W |  15071MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Install the packages we need

In [10]:
# run this cell, then restart the runtime before continuing
!pip install git+https://github.com/joeddav/transformers.git@data-collator-type-fix
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/nlp.git
!pip install transformers
!pip install nlp

  Cloning https://github.com/joeddav/transformers.git (to revision data-collator-type-fix) to /tmp/pip-req-build-dbc5s8ai
  Running command git clone -q https://github.com/joeddav/transformers.git /tmp/pip-req-build-dbc5s8ai
  Running command git checkout -b data-collator-type-fix --track origin/data-collator-type-fix
  Switched to a new branch 'data-collator-type-fix'
  Branch 'data-collator-type-fix' set up to track remote branch 'data-collator-type-fix' from 'origin'.
     |████████████████████████████████| 3.0MB 4.5MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=746347 sha256=16e9bfde41801aa2e0587136bbdc014234a5ffce61538808c2b9433ab59a353e
  Stored in directory: /tmp/pip-ephem-wheel-cache-259lpzbi/wheels/17/d2/a2/c00487983b93ad7c3c487727a842b0f3ea8f7242c89fa93441
Successfully built transformers
  Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Foun

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-4zsctro5
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-4zsctro5
  Using cached https://files.pythonhosted.org/packages/40/d0/30d5f8d221a0ed981a186c8eb986ce1c94e3a6e87f994eae9f4aa5250217/tokenizers-0.8.1rc1-cp36-cp36m-manylinux1_x86_64.whl
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=780998 sha256=641df285ee0591946343265dc4d56faf5b4c9a578e80064ed10c2897164ab3fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-tvgjhova/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers
  Found existing installation: tokenizers 0.8.0rc3
    Uninstalling tokenizers-0.8.0rc3:
      Successfully uninstalled tokenizers-0.8.0rc3
  Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0


  Cloning https://github.com/huggingface/nlp.git to /tmp/pip-req-build-huv435fq
  Running command git clone -q https://github.com/huggingface/nlp.git /tmp/pip-req-build-huv435fq
  Created wheel for nlp: filename=nlp-0.3.0-cp36-none-any.whl size=111969 sha256=41657b0ff3efcdbd5044fc97805f4d9fccf6513839ce832d1096bf0452f7a03d
  Stored in directory: /tmp/pip-ephem-wheel-cache-4kfgwj3b/wheels/a2/98/bd/15f115cb85f1a049a96b915aa14466d3b51a023cd597e08e0d
Successfully built nlp


Import the package we gonna use

In [1]:
# Training
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)
from nlp import load_dataset
import random
import numpy as np

# Evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

Define two functions we gonna use

In [3]:
def compute_metrics(pred):
    """Compute precision, recall, and F1 score.

    Arg:
        pred: The model prediction.
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

def tokenize(batch):
    """Tokenize a batch of data (with padding and truncation).

    Arg:
        batch: A batch of training data.
    """
    return tokenizer(batch["text"], padding=True, truncation=True,)

Load model and tokenizer

In [4]:
# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl

Load dataset and preprocess data

In [5]:
# Load train and test datasets
yelp_train = load_dataset("yelp_polarity", split="train")
yelp_test = load_dataset("yelp_polarity", split="test")

# Randomly choose train and validation indices
# train_indices, val_indices = train_test_split(
#     range(len(yelp_train)), test_size=0.2, train_size=0.8, random_state=random.seed(42)
# )
train_indices, val_indices = train_test_split(
    range(50000), test_size=0.2, train_size=0.8, random_state=random.seed(42)
)

# Split train and validation data
train_dataset = yelp_train.select(indices=train_indices)
val_dataset = yelp_train.select(indices=val_indices)
test_dataset = yelp_test

# Preprocess
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0. Subsequent calls will reuse this data.


100%|██████████| 38/38 [00:06<00:00,  5.63it/s]


Define training arguments and trainer

In [6]:
# Define training arguments
training_args = TrainingArguments(
    adam_epsilon=1e-08,
    eval_steps=10000,
    evaluate_during_training=True,
    gradient_accumulation_steps=1,
    learning_rate=5e-05,
    logging_dir="./logs",
    max_grad_norm=1.0,
    num_train_epochs=1,
    output_dir="./results",
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    save_steps=10000,
    seed=42,
    warmup_steps=0,
    weight_decay=0.0
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

Fine-tune the RoBerta Model

In [7]:
# Fine-tune the model
trainer.train()

TrainOutput(global_step=1250, training_loss=0.1530088638946414)

In [9]:
# Evaluate the model on training set
train_score = trainer.evaluate(eval_dataset=train_dataset)

In [10]:
# Evaluate the model on validation set
val_score = trainer.evaluate(eval_dataset=val_dataset)

In [11]:
print("train_f1:", round(train_score["eval_f1"], 4)) 
print("train_acc:", round(train_score["eval_accuracy"], 4)) 
print("val_f1:", round(val_score["eval_f1"], 4)) 
print("val_acc:", round(val_score["eval_accuracy"], 4)) 

train_f1: 0.977
train_acc: 0.9787
val_f1: 0.9654
val_acc: 0.9673


In [ ]:
# Evaluate the model on test set
test_score = trainer.evaluate(eval_dataset=test_dataset)
print("test_f1:", round(test_score["eval_f1"], 4)) 
print("test_acc:", round(test_score["eval_accuracy"], 4))

Save the model

In [ ]:
trainer.model.save_pretrained("/content/drive/My Drive/models/distilroberta")
tokenizer.save_pretrained("/content/drive/My Drive/models/distilroberta")

Set up the pipline

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Sentiment analysis pipeline
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/models/distilroberta")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/models/distilroberta", use_fast=True)
ppl = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)